# Changes from v11
* Use UMLS to train


# Import the MIMIC data

In [1]:
import gzip
import pandas as pd
import glob

dataset_dictionary = {}

for file_path in glob.glob('.\\Data\\MIMIC Files\*'):
    file_name = file_path.split('\\')[3].split('.')[0]
    with gzip.open(file_path, mode='r') as file:
        dataset_dictionary[file_name] = pd.read_csv(file)

c:\users\amartins\onedrive - intermountain healthcare\python_pycharm_virt_env\.venv\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (4,5,7,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
c:\users\amartins\onedrive - intermountain healthcare\python_pycharm_virt_env\.venv\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Join the tables

In [2]:
# Create dataset to join together -----

# Create note_events table -----

# Combine text for each subject and encounter
note_events_base = dataset_dictionary['NOTEEVENTS'][dataset_dictionary['NOTEEVENTS'].loc[:,'CATEGORY'] == 'Discharge summary']
note_events = note_events_base.groupby(['SUBJECT_ID', 'HADM_ID'], as_index=False)['TEXT'].agg(sum)

# Create CPT table -----

cpt_events_base = dataset_dictionary['CPTEVENTS']
cpt_events_base = cpt_events_base[cpt_events_base['TICKET_ID_SEQ'] == 1]
cpt_events_base = cpt_events_base.loc[:, ['SUBJECT_ID','HADM_ID', 'CPT_CD']]
cpt_events = cpt_events_base.drop_duplicates()
cpt_events

# Join the datasets -----

note_cpt = note_events.merge(cpt_events, on = ['SUBJECT_ID','HADM_ID'])
# print(note_cpt.shape, note_events.shape, cpt_events.shape) # (223,150, 4) (52,726, 3) (227,510, 3)

# Filter the dataset to those CPT codes over 200 notes

In [3]:
import numpy as np
from sklearn.utils import resample

df = note_cpt['CPT_CD'].astype(str).value_counts()
top_200 = list((df[df > 200]).index.values)
note_cpt_200 = note_cpt[note_cpt['CPT_CD'].astype(str).isin(top_200)]

# Filter the data

In [7]:
import string

def clean_data(text_series):
    
    # Replace \n 
    text_series = text_series.str.replace('\\n',' ', regex=True)    

    # Remove dates and locations
    text_series = text_series.str.replace('\[\*\*(.*?)\*\*\]', ' ', regex=True)
    
    # Remove topics
    data = text_series.str.split('([A-Z\s]+:)')
    for row_num, value in enumerate(data):
        text_chunks = [x.strip().replace(':','').replace('\n', '') for x in value]
        for i, x in enumerate(text_chunks):
            if 'MEDICATION' in x or 'SOCIAL HISTORY' in x or 'FAMILY HISTORY' in x:
                text_chunks[i] = ' '
                try:
                    text_chunks[i + 1] = ' '
                except:
                    continue

        text_series.iloc[row_num] = ' '.join(text_chunks)
    
    # Replace punctuation
    text_series = text_series.str.replace('[' + string.punctuation + ']', ' ', regex=True)
    
    # Convert to lowercase 
    text_series = text_series.str.lower()
    
    # Remove all digits
    text_series = text_series.str.replace('\d',' ', regex=True)
    
    # Replace plurals, endings with ing, endings with ed, endings with ly
#     text_series = text_series.str.replace('s(?=\s)', ' ', regex=True)
#     text_series = text_series.str.replace('ing(?=\s)', ' ', regex=True)
#     text_series = text_series.str.replace('ed(?=\s)', ' ', regex=True)
#     text_series = text_series.str.replace('ly(?=\s)', ' ', regex=True)
    
    return text_series

# Update Text Column

note_cpt_200.loc[:, 'TEXT'] = clean_data(note_cpt_200['TEXT']).values

# for i in Sample_Metamap.keys():
#     Sample_Metamap[i]['TEXT'] = clean_data(Sample_Metamap[i]['TEXT'])

c:\users\amartins\onedrive - intermountain healthcare\python_pycharm_virt_env\.venv\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [12]:
len(note_cpt_200)

19935

# Loop through and Create Txt Files from Df

In [11]:
import numpy as np

note_cpt_200 = note_cpt_200.reset_index()
for index in range(len(note_cpt_200)):
    with open('CPT Text Files\\' + str(note_cpt_200.loc[index, 'CPT_CD']) + '_' + str(index) + '.txt', "w") as text_file:
        text_file.write(note_cpt_200.loc[index, 'TEXT'])

# Import All the XMI Files

In [ ]:
import glob
import re

procedure_ls = []

proc_regex = re.compile('(?<="T061"\spreferredText=")[\w+\s,]*')

for i in glob.glob('.\\*.xmi'):
    with open(i) as file:
        file_contents = file.read()
        procedures = proc_regex.findall(file_contents)
        procedure_ls.append(list(set(procedures)))

# Filter the data to CPT over 200 samples + Resample

In [92]:
import numpy as np
from sklearn.utils import resample

# Value Counts
print(note_cpt['CPT_CD'].astype(str).value_counts())

# Filter to CPT with over 200 notes
df = note_cpt['CPT_CD'].astype(str).value_counts()
top_200 = list((df[df > 200]).index.values)
note_cpt_4 = note_cpt[note_cpt['CPT_CD'].astype(str).isin(top_200)]

# Resample minority groups -----

# Remove largest group
top_200.remove('99291')
'99291' in top_200

minority_ls = top_200
# minority_ls = ['99223','99222','99254']

minority_df = []
for i in minority_ls:
    test_resampled = resample(note_cpt[note_cpt['CPT_CD'].astype(str) == i], replace=True, n_samples=7860, random_state=123)
    minority_df.append(test_resampled)

minority_df.append(note_cpt[note_cpt['CPT_CD'].astype(str) == '99291'])
new_df = pd.concat(minority_df)

new_df['CPT_CD'] = new_df['CPT_CD'].astype(str)

99291    7860
99223    2851
99222    1736
99254    1242
99255     882
         ... 
33542       1
22630       1
26418       1
63103       1
56620       1
Name: CPT_CD, Length: 707, dtype: int64


# Check for Imbalance

In [93]:
# import matplotlib.pyplot as plt

# plt.hist(note_cpt['CPT_CD'].astype(str))
# plt.show()

new_df['CPT_CD'].value_counts()

99252    7860
33508    7860
99222    7860
99291    7860
99253    7860
99233    7860
99254    7860
99231    7860
33405    7860
99292    7860
33533    7860
99255    7860
36556    7860
99221    7860
99232    7860
99223    7860
Name: CPT_CD, dtype: int64

In [59]:
for i in Sample_Metamap.keys():
    Sample_Metamap[i].to_csv(i + '.csv')

# Shuffle the Data

In [7]:
new_df = new_df.sample(n = len(new_df), random_state = 42)

# Split the Data

In [373]:
# Import Packages -----

from nltk.corpus import stopwords

my_stop_words = list(set(stopwords.words('english'))) \
                + ['admission', 'date', 'sex'] \
                + ['needed', 'every', 'seen', 'weeks', 'please', 'ml', 'unit', 'small', 'year', 'old', 'cm', 'non', 'mm', 'however']
                # Got the above from my top 100 most predictive words that I wanted to remove

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Split the data -----

X_train, X_test, y_train, y_test = train_test_split(z[z['CPT_CD'].astype(str).isin(['99221','36556', '99291'])][2].values, z[z['CPT_CD'].astype(str).isin(['99221','36556', '33518'])]['CPT_CD'].astype(str), test_size = .1, random_state = 42)

# Tokenize the data

In [374]:
# Tokenize the data -----

# Import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words=my_stop_words)

# Transform the training data
tfidf_train = tfidf_vectorizer.fit_transform(X_train)

# Transform the test data
tfidf_test = tfidf_vectorizer.transform(X_test)

# Run Naive Bayes

In [375]:
# Use Naive Bayes model -----

from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

nb_classifier = MultinomialNB(alpha=.7)

# Fit and check accuracy
nb_classifier.fit(tfidf_train, y_train)
pred = nb_classifier.predict(tfidf_test)

# Looking at Feature Names and Weights

In [167]:
# Notes
# sum([np.exp(1)** x for x in nb_classifier.coef_[0]]) # The probability of all the words equals one
# # Taken from here: * https://stackoverflow.com/questions/61586946/how-to-calculate-feature-log-prob-in-the-naive-bayes-multinomialnb

# ------------------------------------------

import numpy as np

def get_feature_rank(tfidf_vectorizer, y_no, nb_classifier):
    
    # Get the feature names
    feature_names = tfidf_vectorizer.get_feature_names()

    # Zip together the first CPT weights with feature names
    feat_with_weights =  sorted(zip(nb_classifier.coef_[y_no], feature_names))
    
    # Print words most responsible for the prediction
#     print('Top 100 \n\n\n\n')
#     top_100_ls = []
    for i in range(100):
        x = feat_with_weights[-i-1]
#         top_100_ls.append(x[1])
#         print(nb_classifier.classes_[y_no], i, round((np.exp(1) ** x[0]),4), x[1])

#     print('\n\n\n\n Bottom 100 \n\n\n\n')
    for i in range(100):
        x = feat_with_weights[i]
#         print(nb_classifier.classes_[y_no], i, round((np.exp(1) ** x[0]),4), x[1])
    
#     min_weight = min([i[0] for i in feat_with_weights])
    
    x = [i[0] for i in feat_with_weights]
    
    median_pred = np.median(x)
          
    return [i[1] for i in feat_with_weights if i[0] <= median_pred] # Minimum weight words
#     return top_100_ls

# Find the least predictive words
def least_pred_words(nb_classifier, tfidf_vectorizer):
    low_wt_stop_ls = []

    for i in range(len(nb_classifier.classes_)):
        low_wt_stop_ls += get_feature_rank(tfidf_vectorizer, i, nb_classifier)

    low_wt_stop_ls = list(set(low_wt_stop_ls))
    return low_wt_stop_ls
    
low_wt_stop_ls = least_pred_words(nb_classifier, tfidf_vectorizer)

# Find top 100 words - doesn't seem to improve the model
def highest_pred_words(nb_classifier, tfidf_vectorizer):
    top_100_ls = []
    for i in range(len(nb_classifier.classes_)):
        top_100_ls += get_feature_rank(tfidf_vectorizer, i, nb_classifier)

    top_100_ls = list(set(top_100_ls))
    return top_100_ls

['citizenship',
 'inutbag',
 'reamin',
 'taspe',
 'traycan',
 'eleveat',
 'interscapular',
 'stenossis',
 'spp',
 'tct',
 'resectable',
 'throacotomy',
 'evisiting',
 'nannulu',
 'tplt',
 'arthrotec',
 'asberger',
 'nnor',
 'sttw',
 'inconssitent',
 'contemplate',
 'whofelt',
 'nreporttemporal',
 'femaile',
 'nbengay',
 'synoviti',
 'flie',
 'controls',
 'navalide',
 'nlipomatosi',
 'endeavour',
 'tractor',
 'expectorated',
 'tegretal',
 'nvalproate',
 'walnut',
 'ncarie',
 'fulminan',
 'nrole',
 'shariro',
 'duckbill',
 'nviridan',
 'supression',
 'mwt',
 'pseudobulbar',
 'propecia',
 'nqpm',
 'kindey',
 'narrowed',
 'maex',
 'inheritable',
 'speculum',
 'intracran',
 'otsc',
 'chornical',
 'nvetriculography',
 'nhycodan',
 'abcces',
 'hypotenion',
 'blank',
 'nfile',
 'herniorrhaphie',
 'nklonipin',
 'ntelapivir',
 'sfebrile',
 'nafld',
 'nkiss',
 'falumouth',
 'hematoemesi',
 'wrote',
 'nspecgr',
 'npostoop',
 'repond',
 'nnwh',
 'reliever',
 'nresultant',
 'nmeasures',
 'digibind',

# Classification Report

In [368]:
# Create classification report taken from here: https://towardsdatascience.com/multi-class-text-classification-model-comparison-and-selection-5eb066197568
# from sklearn.metrics import classification_report

print('Test')
class_labels = nb_classifier.classes_
print(classification_report(y_test, pred,target_names=class_labels))

print('Training')
pred_x = nb_classifier.predict(tfidf_train)
print(classification_report(y_train, pred_x,target_names=class_labels))

print(y_test)
print(pred)
print(y_train)

print(metrics.accuracy_score(y_test, pred))
print(y_test, pred)


Test


ValueError: Number of classes, 2, does not match size of target_names, 3. Try specifying the labels parameter